# ADK 101 in 45min


Where to test:
- stdout output
- `adk web` as UI -- very nice visualizations

Pre-caution:
- ADK is still under development. The official launch is early April. This training is intended to provide hands-on experience with the Agent Development Kit (ADK) and its foundational concepts.


Documents 
https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/use/adk



# Environment


In [1]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# !pip3 install google_adk 
# !pip install -q --upgrade google-cloud-discoveryengine
# !pip install google-adk
# !pip install google-genai
# !pip install pydantic

#-0.0.2.dev20250324+739344859-py3-none-any.whl

In [3]:
import os

# Cloud project id.
PROJECT_IDS = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_IDS[0]  # @param {type:"string"}

if not PROJECT_ID:
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = "us-central1" # @param {type:"string"}

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE" # Use Vertex AI API
# [your-project-id]

In [62]:
from google.cloud import storage

client = storage.Client()

GCS_BUCKET_LOCATION = LOCATION
UNIQUE_PREFIX = "sm"

GCS_BUCKET_NAME = f"{PROJECT_ID}-{UNIQUE_PREFIX}"
GCS_BUCKET_URI = f"gs://{GCS_BUCKET_NAME}"

bucket = storage.Bucket(client, GCS_BUCKET_NAME)

if bucket.exists()==False:
    # Create a Cloud Storage Bucket
    !gcloud storage buckets create $GCS_BUCKET_URI --location=$GCS_BUCKET_LOCATION

else:
    # Upload the PDFs located in the books/ directory into the GCS bucket that you created
    
    print(f"\n{GCS_BUCKET_NAME} already exists. Contents:\n")
    
def gcs_file(blob_name):
    return bucket.blob(blob_name)


Creating gs://my-project-0004-346516-sm/...
CommandException: No URLs matched: /books/*


In [4]:
import vertexai
from vertexai import agent_engines
from vertexai.preview.reasoning_engines import AdkApp

LOCATION = "us-central1" #@param {type:"string"}
STAGING_BUCKET = GCS_BUCKET_URI

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

In [5]:
# !gcloud storage ls gs://adk_training/sdk
# !gcloud storage cp gs://adk_training/sdk/google_adk-0.0.2.dev20250324+739344859-py3-none-any.whl .

In [6]:
import random

from google.adk.runners import InMemoryRunner, Runner
from google.adk.agents import BaseAgent, LlmAgent, Agent, SequentialAgent, LoopAgent, ParallelAgent
from google.adk.agents.readonly_context import ReadonlyContext
from google.adk.tools import ToolContext, LongRunningFunctionTool
from google.adk.tools.agent_tool import AgentTool
from google.adk.sessions import Session
from google.adk.events import Event
from google.adk.agents.callback_context import CallbackContext

from google.genai import types

from pydantic import BaseModel, Field
from typing import List, Optional
import logging

logging.basicConfig(level=logging.ERROR)

# Utils

In [7]:
import json
import time

def pprint_events(events):
    '''Pretty print of events generated by ADK runner'''
    start_time = time.time()

    for _, event in enumerate(events):
        is_final_response = event.is_final_response()
        function_calls = event.get_function_calls()
        function_responses = event.get_function_responses()

        try:
            agent_res = json.loads(event.content.model_dump_json(indent=2, exclude_none=True))
        except AttributeError as e:
            print(f"Error parsing event content: {e}")
            continue

        if is_final_response:
            final_response = event.content.parts[0].text if event.content.parts else "No content available"
            elapsed_time_ms = round((time.time() - start_time) * 1000, 3)
            print(f'>>> Final Response ({elapsed_time_ms} ms):\n{final_response}')
            print("-" * 30)
        elif function_calls:
            print('+++ Function Calls:')
            for function_call in function_calls:
                print(f"Function Name: {function_call.name}, Args: {function_call.args}")
        elif function_responses:
            print('--- Function Responses:')
            for function_response in function_responses:
                response_details = function_response.response
                recommended_list = list(response_details.values()) if response_details else []
                print(f"Function Name: {function_response.name}")
                print(f"Function Results: {json.dumps(recommended_list)}")
        else:
            print('No function calls or responses available.')
            print(f"Agent Response: {agent_res}")

    elapsed_time_ms = round((time.time() - start_time) * 1000, 3)
    print(f"Total elapsed time: {elapsed_time_ms} ms")


In [8]:
from typing import Optional, Any

APP_NAME = 'test_app'
USER_ID = 'test_user'

def create_runner(agent: BaseAgent):
  return InMemoryRunner(agent, app_name=APP_NAME)

def _content_to_text(content: types.Content | None) -> str:
  if not content or not content.parts:
    return ''
  return ''.join([p.text or '' for p in content.parts])

def print_event(event: Event):
  print(f'Author: {event.author}')
  print(f'Content Text: {_content_to_text(event.content)}')
  print(f'Event: {event.model_dump(exclude_none=True, exclude_defaults=True)}')

async def run_session(new_message: types.Content, *, runner: Runner, session: Optional[Session] = None, state: Optional[dict[str, Any]] = None) -> Session:
  if session is None:
    session = runner.session_service.create_session(app_name=APP_NAME, user_id=USER_ID, state=state)

  print(f'User: {_content_to_text(new_message)}')
  print('----------------------------------')
  async for e in runner.run_async(user_id=USER_ID, session_id=session.id, new_message=new_message):
    print_event(e)
  print('----------------------------------')

  session = runner.session_service.get_session(app_name=session.app_name, user_id=session.user_id, session_id=session.id)
  return session

def content_text(msg: str) -> types.Content:
  return types.UserContent(parts=[types.Part(text=msg)])

# Hello world!

Concepts:

- Agent - Starts as a wrapper of LLM that can decide actions and interact with external world (other agents, tools)
- Session - Maintains the state of one invocation. (chat log / events/ etc)
- Runner - The orchestration layer of agents. The "state machine". Runner drives agents to move the states of a session forward.

Use case:
- An agent that says hello word in a random language

In [9]:
from google.genai import types
from pydantic import BaseModel
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.agents import LlmAgent
from google.adk.tools.agent_tool import AgentTool
from google.adk.sessions import InMemorySessionService

import warnings
import logging
from pprint import pprint

logging.getLogger('google_genai.types').setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=UserWarning, module='google.generativeai.types.content_types') # Suppress harmless warning

In [10]:
# Constant
APP_NAME = "hello_world_example"
USER_ID = "user12345"
SESSION_ID = "session12345"
AGENT_NAME = "hello_word_agent"
MODEL = "gemini-2.0-flash-001"

**Instructions:** Start by defining what your agent should do using simple natural language instructions. This provides a very straightforward way to customize agent actions without needing to compose complex chains or graphs. You can even reference context like user information and current time within these instructions, which will be filled in at runtime




**Sessions:** The Agent Framework handles session management for you, allowing you to focus on building your agent's logic. You can use the provided BaseSession interface with managed storage or easily plug in your own. The InMemorySession is available for rapid development and debugging

In [11]:
# Agent
hello_world_agent = Agent(
    model=MODEL,
    name="hello_world_agent",
    description="An agent that says 'hello world'",
    instruction="""You always say 'hello world' to the user, and nothing else.
    Output 'hello world' in a random language.
    Put the language in brackets.

    Example Output 1:
    hello world (English)

    Example Output 2:
    你好，世界 (Chinese)
    """,
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=100,
    ),
)

# Session and Runner
session_service = InMemorySessionService()
session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=hello_world_agent, app_name=APP_NAME, session_service=session_service)

# Agent Interaction
def call_agent(runner, query):
  content = types.Content(role='user', parts=[types.Part(text=query)])
  events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)
  return events

In [12]:
events = call_agent(runner, "hello")
pprint_events(events)

>>> Final Response (1102.272 ms):
Hola mundo (Spanish)

------------------------------
Total elapsed time: 1103.703 ms


In [13]:
session

Session(id='session12345', app_name='hello_world_example', user_id='user12345', state={}, events=[], last_update_time=1748497092.7940357)

In [14]:
session.id

'session12345'

In [15]:
session_service.list_sessions(app_name=APP_NAME, user_id=USER_ID)

#TypeError: InMemorySessionService.list_sessions() missing 2 required keyword-only arguments: 'app_name' and 'user_id'


ListSessionsResponse(sessions=[Session(id='session12345', app_name='hello_world_example', user_id='user12345', state={}, events=[], last_update_time=1748497092.801006)])

In [16]:
pprint(session_service.sessions[APP_NAME][USER_ID][SESSION_ID].events)

[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='hello')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, invocation_id='e-b3ab7b22-bda7-4526-929d-f5c62003e987', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='gGbkwLla', timestamp=1748497092.800294),
 Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='Hola mundo (Spanish)\n')], role='model'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_mes

# Multi turn conversation

Concepts:

- Event(s)
- Session
- SessionService: sessions[app][user][session]

Use case:

- An agent that tries to know your name and send hello USERNAME

In [17]:
# Constant
APP_NAME = "hello_name_example"
USER_ID = "user12345"
SESSION_ID = "session12345"
AGENT_NAME = "hello_name_agent"
MODEL = "gemini-2.0-flash-001"

# Agent
hello_name_agent = Agent(
    model=MODEL,
    name="hello_name_agent",
    description="An agent that says 'hello USERNAME'",
    instruction="""
    You need to first ask the user's name.
    Try best to convince the user to give you a name, let it be first name, last name, or nick name.

    Once you get the user's name, say 'hello USERNAME'.
    """,
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=100,
    ),
)

# Session and Runner
session_service = InMemorySessionService()
session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=hello_name_agent, app_name=APP_NAME, session_service=session_service)

# Agent Interaction
def call_agent(runner, session, query):
  content = types.Content(role='user', parts=[types.Part(text=query)])
  events = runner.run(user_id=session.user_id, session_id=session.id, new_message=content)
  return events

events = call_agent(runner, session, "hello")
pprint_events(events)

>>> Final Response (1247.231 ms):
Hello there! Before I say hello back, could you tell me what name I should use? It can be your first name, last name, or even a nickname! I just want to be a bit more personal. 😊

------------------------------
Total elapsed time: 1248.592 ms


In [18]:
events = call_agent(runner, session, "I don't tell you my name")
pprint_events(events)

>>> Final Response (1382.28 ms):
I understand, some people prefer to keep their name private. But just between us, wouldn't it be nice if I could greet you properly? Even a simple nickname would do! It would make our conversation feel a bit more friendly. Pretty please? 😊

------------------------------
Total elapsed time: 1383.649 ms


In [19]:
events = call_agent(runner, session, "what is your name?")
pprint_events(events)

>>> Final Response (1327.491 ms):
You can call me hello_name_agent! But I'd still love to know what I can call you! Think of it as a little secret between us. 😉

------------------------------
Total elapsed time: 1328.769 ms


In [20]:
events = call_agent(runner, session, "my name is Amir")
pprint_events(events)

>>> Final Response (927.052 ms):
Hello Amir! It's nice to meet you!

------------------------------
Total elapsed time: 928.329 ms


In [21]:
events = call_agent(runner, session, "what is my name?")
pprint_events(events)

>>> Final Response (1032.633 ms):
Your name is Amir! I'll try my best to remember that. 😊

------------------------------
Total elapsed time: 1034.099 ms


## Switch session

In [22]:
new_session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID + '-new')

In [23]:
events = call_agent(runner, new_session, "what is my name?")
pprint_events(events)

>>> Final Response (1055.094 ms):
I don't know your name yet. Could you please tell me what I should call you? Even a nickname would be fine!

------------------------------
Total elapsed time: 1056.501 ms


In [24]:
events = call_agent(runner, session, "what is my name?")
pprint_events(events)

>>> Final Response (1041.076 ms):
You told me your name is Amir! I'll do my best not to forget. 😊

------------------------------
Total elapsed time: 1042.395 ms


# Use Tool

**Function Tools:** To extend your agent's capabilities, you can provide it with Python functions as tools. When the agent determines it needs to perform a specific action, it can automatically call these functions

Concept:
- Tool (Python function) -- the experience of tool calling by agent (by LLM) or by human (from Python) are the same. Easy to test.

Use case:
- Simple math

In [25]:
def add(numbers: list[int]) -> int:
  """Calculates the sum of a list of integers.

    This function takes a list of integers as input and returns the sum of all
    the elements in the list.  It uses the built-in `sum()` function for
    efficiency.

    Args:
        numbers: A list of integers to be added.

    Returns:
        The sum of the integers in the input list.  Returns 0 if the input
        list is empty.

    Examples:
        add([1, 2, 3]) == 6
        add([-1, 0, 1]) == 0
        add([]) == 0
    """
  return sum(numbers)

def subtract(numbers: list[int]) -> int:
    """Subtracts numbers in a list sequentially from left to right.

    This function performs subtraction on a list of integers, applying the
    subtraction operation from left to right.  For example, given the list
    [10, 2, 5], the function will calculate 10 - 2 - 5.

    Args:
        numbers: A list of integers to be subtracted.

    Returns:
        The result of the sequential subtraction as an integer. Returns 0 if the input list is empty.

    Examples:
        subtract([10, 2, 5]) == 3  # (10 - 2) - 5 = 8 - 5 = 3
        subtract([10, 2]) == 8      # 10 - 2 = 8
        subtract([]) == 0
    """
    if not numbers:
        return 0  # Handle empty list
    result = numbers[0]
    for num in numbers[1:]:
        result -= num
    return result

def multiply(numbers: list[int]) -> int:
  """Calculates the product of a list of integers.

    This function takes a list of integers as input and returns the product of all
    the elements in the list. It iterates through the list, multiplying each
    number with the accumulated product.

    Args:
        numbers: A list of integers to be multiplied.

    Returns:
        The product of the integers in the input list. Returns 1 if the input
        list is empty.

    Examples:
        multiply([2, 3, 4]) == 24  # 2 * 3 * 4 = 24
        multiply([1, -2, 5]) == -10 # 1 * -2 * 5 = -10
        multiply([]) == 1
    """
  product = 1
  for num in numbers:
    product *= num
  return product

def divide(numbers: list[int]) -> float:  # Use float for division
    """Divides numbers in a list sequentially from left to right.

    This function performs division on a list of integers, applying the division
    operation from left to right.  For example, given the list [10, 2, 5], the
    function will calculate 10 / 2 / 5.

    Args:
        numbers: A list of integers to be divided.

    Returns:
        The result of the sequential division as a float.

    Raises:
        ZeroDivisionError: If any number in the list *after* the first element
                           is zero, a ZeroDivisionError is raised.  Division by
                           zero is not permitted.

    Returns:
        float: The result of the division. Returns 0.0 if the input list is empty.

    Examples:
        divide([10, 2, 5]) == 1.0  # (10 / 2) / 5 = 5 / 5 = 1.0
        divide([10, 2]) == 5.0      # 10 / 2 = 5.0
        divide([10, 0, 5])  # Raises ZeroDivisionError
        divide([]) == 0.0
    """
    if not numbers:
        return 0.0 # Handle empty list
    if 0 in numbers[1:]: # Check for division by zero
        raise ZeroDivisionError("Cannot divide by zero.")
    result = numbers[0]
    for num in numbers[1:]:
        result /= num
    return result


In [26]:
add([2, 2, 3])

7

In [27]:
multiply([2, 2, 3])

12

In [28]:
simple_math_agent = Agent(
    model=MODEL,
    name="simple_math_agent",
    description="This agent performs basic arithmetic operations (addition, subtraction, multiplication, and division) on user-provided numbers, including ranges.",
    instruction="""
      I can perform addition, subtraction, multiplication, and division operations on numbers you provide.
      Tell me the numbers you want to operate on.
      For example, you can say 'add 3 5', 'multiply 2, 4 and 3', 'Subtract 10 from 20', 'Divide 10 by 2'.
      You can also provide a range: 'Multiply the numbers between 1 and 10'.
    """,
    generate_content_config=types.GenerateContentConfig(temperature=0.2),
    tools=[add, subtract, multiply, divide],
)

In [29]:
import random

def caller_factory(root_agent, app_name='App12345', user_id='User12345', session_id=None):
  """create a pre-configured agent caller.

    Args:
        root_agent: The ADK agent to handle conversations
        app_name: Application name (default: 'App12345')
        user_id: User identifier (default: 'User12345')
        session_id: Optional session ID. If None, generates a random one.

    Returns:
        A function that takes a query string and returns agent response events.
  """
  session_service = InMemorySessionService() #  to manage conversation sessions
  if session_id is None:
    suffix = random.randint(100000, 999999)
    session_id = f'{app_name}-{user_id}-{suffix}'
  session = session_service.create_session(
      app_name=app_name, user_id=user_id, session_id=session_id)
  runner = Runner(agent=root_agent, app_name=app_name, session_service=session_service) #Create a Runner instance that will execute the agent
  def _call(query):
    content = types.Content(role='user', parts=[types.Part(text=query)])
    events = runner.run(user_id=session.user_id, session_id=session.id, new_message=content)
    return events
  return _call

In [30]:
call = caller_factory(root_agent=simple_math_agent)

In [31]:
pprint_events(call('hello'))

>>> Final Response (1018.92 ms):
Hello! How can I help you with your math today?

------------------------------
Total elapsed time: 1020.145 ms


In [32]:
pprint_events(call('what is three plus 9?'))

+++ Function Calls:
Function Name: add, Args: {'numbers': [3, 9]}
--- Function Responses:
Function Name: add
Function Results: [12]
>>> Final Response (2514.23 ms):
The answer is 12.

------------------------------
Total elapsed time: 2515.634 ms


In [33]:
pprint_events(call('multiply that by 2'))

+++ Function Calls:
Function Name: multiply, Args: {'numbers': [12, 2]}
--- Function Responses:
Function Name: multiply
Function Results: [24]
>>> Final Response (1977.303 ms):
The answer is 24.

------------------------------
Total elapsed time: 1978.478 ms


In [34]:
call = caller_factory(root_agent=simple_math_agent)
pprint_events(call('Here is my math problem that is about apple counting. Let us start saying that I have three apple'))

>>> Final Response (1217.662 ms):
Okay. I understand you have three apples. What would you like to do with that number? Do you want to add more apples, subtract some, multiply by a certain amount, or divide them somehow?

------------------------------
Total elapsed time: 1218.966 ms


In [35]:
pprint_events(call('Alice gave anoter 2 apples'))

>>> Final Response (1010.53 ms):
Okay, so you want to add 2 apples to your existing 3 apples. Is that correct?

------------------------------
Total elapsed time: 1011.751 ms


In [36]:
pprint_events(call('how many apples do I have?'))

+++ Function Calls:
Function Name: add, Args: {'numbers': [3, 2]}
--- Function Responses:
Function Name: add
Function Results: [5]
>>> Final Response (1962.844 ms):
You have 5 apples.

------------------------------
Total elapsed time: 1964.326 ms


In [37]:
pprint_events(call('Bob gave me 3 apples. how many apples do I have?'))

+++ Function Calls:
Function Name: add, Args: {'numbers': [5, 3]}
--- Function Responses:
Function Name: add
Function Results: [8]
>>> Final Response (1929.419 ms):
You have 8 apples.

------------------------------
Total elapsed time: 1930.629 ms


In [38]:
pprint_events(call('Alice and Bob do this to me for three more times. how many apples do I have?'))

+++ Function Calls:
Function Name: add, Args: {'numbers': [8, 15]}
--- Function Responses:
Function Name: add
Function Results: [23]
>>> Final Response (2659.871 ms):
You will have 23 apples.

------------------------------
Total elapsed time: 2661.265 ms


In [39]:
pprint_events(call('yes your logic is correct'))

>>> Final Response (988.407 ms):
Great! I'm glad I could help.

------------------------------
Total elapsed time: 989.779 ms


# Agent As Tool

**AgentTool:** You can embed the power of one agent within another by using AgentTool. This allows you to treat an entire agent as a tool within a parent agent. AgentTools are executed in an isolated environment, promoting safety and modularity. Combined with input/output schemas, this enables the creation of sophisticated interactions


Function Tool vs Agent Tool:
 If the execution flow goes from Agent to Tool and it always come back to theinitiation agent vs if the execution flow goes from Agent to **another** agent, the latter agent can decide where the flow goes next.

Concepts:
- AgentTool -- If you want another agent to handle a task and always come back to the caller, make this `Agent` a `Tool` by `AgentTool`

Use case:
- Advanced math agent

In [40]:
agent_math_advanced_instruction = '''
I am an advanced math agent. I handle user query in the below steps:

1. I shall analyse the chat log to understand current question and make a math formula for it.
2. Break down a complex compuation based on arithmetic priority and hand over to simple_math_agent for the calculation.
3. Note that simple_math_agent can only understand numbers, so I need to convert natural language expression of numbers into digits.

<example>
<input> alice gives us 3 apples, bob gives us 5 apples. They do this seven times. Then we eat four apples. How many apples do we have now? </input>
<think> what is (3+5) * 7 -4 </think>
<think>I need to first calculate (3+5) as the highest priority operation.</think>
<call_tool> pass (3+5) to simple_math_agent </call_tool>
<tool_response>8</tool_response>
<think> The question now becomes 8 * 7 - 4, and next highest operation is 8 * 7</think>
<call_tool> pass 8 * 7 to simple_math_agent </call_tool>
<tool_response>56</tool_response>
<think> The question now becomes 56 - 4, and next highest operation is 56 - 4</think>
<call_tool> pass 56 - 4 to simple_math_agent </call_tool>
<tool_response>52</tool_response>
<think>There is a single number, so it is the final answer.</think>
<output>The result of "(3+5) * 7 - 4" is 52</output>
</example>
'''

agent_math_advanced = Agent(
    model=MODEL,
    name="agent_math_advanced",
    description="The advanced math agent can break down a complex computation into multiple simple operations and use math_agent to solve them.",
    instruction=agent_math_advanced_instruction,
    tools=[AgentTool(agent=simple_math_agent)],
    generate_content_config=types.GenerateContentConfig(temperature=0.2),
)

In [41]:
call = caller_factory(root_agent=agent_math_advanced)

In [42]:
pprint_events(call('who are you?'))

>>> Final Response (1263.15 ms):
I am an advanced math agent. I handle user query in the below steps:

1. I shall analyse the chat log to understand current question and make a math formula for it.
2. Break down a complex compuation based on arithmetic priority and hand over to simple_math_agent for the calculation.
3. Note that simple_math_agent can only understand numbers, so I need to convert natural language expression of numbers into digits.

------------------------------
Total elapsed time: 1264.408 ms


In [43]:
pprint_events(call('what is 1 + (three+2) times 7 '))

+++ Function Calls:
Function Name: simple_math_agent, Args: {'request': '3+2'}
--- Function Responses:
Function Name: simple_math_agent
Function Results: ["The answer is 5.\n"]
+++ Function Calls:
Function Name: simple_math_agent, Args: {'request': '5*7'}
--- Function Responses:
Function Name: simple_math_agent
Function Results: ["35\n"]
+++ Function Calls:
Function Name: simple_math_agent, Args: {'request': '1+35'}
--- Function Responses:
Function Name: simple_math_agent
Function Results: ["36\n"]
>>> Final Response (10302.9 ms):
The result of "1 + (three+2) times 7" is 36

------------------------------
Total elapsed time: 10304.296 ms


In [44]:
call = caller_factory(root_agent=agent_math_advanced)

In [45]:
pprint_events(call('add from 1 to 2. then multiply that by 2.'))

+++ Function Calls:
Function Name: simple_math_agent, Args: {'request': '1+2'}
--- Function Responses:
Function Name: simple_math_agent
Function Results: ["3\n"]
+++ Function Calls:
Function Name: simple_math_agent, Args: {'request': '3*2'}
--- Function Responses:
Function Name: simple_math_agent
Function Results: ["The answer is 6.\n"]
>>> Final Response (7036.883 ms):
The result of "add from 1 to 2. then multiply that by 2" is 6.

------------------------------
Total elapsed time: 7038.321 ms


In [46]:
call = caller_factory(root_agent=agent_math_advanced)
pprint_events(call('how much is 10 / 5 + three times 5?'))

+++ Function Calls:
Function Name: simple_math_agent, Args: {'request': '10 / 5'}
--- Function Responses:
Function Name: simple_math_agent
Function Results: ["The answer is 2.0\n"]
+++ Function Calls:
Function Name: simple_math_agent, Args: {'request': '3 * 5'}
--- Function Responses:
Function Name: simple_math_agent
Function Results: ["15\n"]
+++ Function Calls:
Function Name: simple_math_agent, Args: {'request': '2.0 + 15'}
--- Function Responses:
Function Name: simple_math_agent
Function Results: ["I am designed to work with integers. Could you provide me with integers only?\n"]
+++ Function Calls:
Function Name: simple_math_agent, Args: {'request': '2 + 15'}
--- Function Responses:
Function Name: simple_math_agent
Function Results: ["The answer is 17."]
>>> Final Response (12759.08 ms):
The result of "10 / 5 + three times 5" is 17

------------------------------
Total elapsed time: 12760.399 ms


**NOTE**: This part is very nice, it shows how two agents interact to clarify the input type should be integers and input again.

```
+++ inside the function call...
function, [args]:  simple_math_agent, {'request': '2.0 + 15'}
--- inside the function call response...
Function Name: simple_math_agent
Function Results ["I can only work with integers. Could you provide the numbers as integers?\n"]
WARNING:google_genai.types:Warning: there are non-text parts in the response: ['function_call'],returning concatenated text from text parts,check out the non text parts for full response from model.
+++ inside the function call...
function, [args]:  simple_math_agent, {'request': '2 + 15'}

```

# Input/Output Format Control

**Input Schema / Output Schema:** Ensure data consistency and validity by defining Pydantic schemas for your agent's input and output. The agent can then verify the input against the schema and guarantee that the generated output conforms to the specified structure using constrained decoding

In [47]:
from typing import List
from pydantic import BaseModel, Field
from google.genai import types

json_response_config = types.GenerateContentConfig(
  response_mime_type="application/json",
)

class OutputSchema(BaseModel):
    original_query: str = Field(description="The original text from user.")
    corrected_text: str = Field(description="The corrected text.")
    errors: List[str] = Field(description="An array of descriptions of each error.")
    explanations: List[str] = Field(description="An array of explanations for each correction.")

json_schema = OutputSchema.model_json_schema()
json_schema

{'properties': {'original_query': {'description': 'The original text from user.',
   'title': 'Original Query',
   'type': 'string'},
  'corrected_text': {'description': 'The corrected text.',
   'title': 'Corrected Text',
   'type': 'string'},
  'errors': {'description': 'An array of descriptions of each error.',
   'items': {'type': 'string'},
   'title': 'Errors',
   'type': 'array'},
  'explanations': {'description': 'An array of explanations for each correction.',
   'items': {'type': 'string'},
   'title': 'Explanations',
   'type': 'array'}},
 'required': ['original_query', 'corrected_text', 'errors', 'explanations'],
 'title': 'OutputSchema',
 'type': 'object'}

In [48]:
# 1. The {json_schema} in instruction is the key for model to follow the schema.
# 2. The output_schema=OutputSchema provides a validation step after model output.

agent_grammar = Agent(
    model=MODEL,
    name='agent_grammar',
    description="This agent corrects grammar mistakes in text provided by children, explains the errors in simple terms, and returns both the corrected text and the explanations.",
    instruction=f"""
        You are a friendly grammar helper for kids.  Analyze the following text,
        correct any grammar mistakes, and explain the errors in a way that a
        child can easily understand.  Don't just list the errors; explain them
        in a paragraph using simple but concise language.

        Output in a JSON object with the below schema:
        {json_schema}
    """,
    output_schema=OutputSchema,
    generate_content_config=json_response_config,
    disallow_transfer_to_parent = True,
    disallow_transfer_to_peers=True
    
    # allow_transfer=False,
)

Conversation Flow:

The session contains multiple interactions where the user asks questions with intentional misspellings, and the agent corrects them. Here are the main interactions:

Response Format: Each agent response is in JSON format containing:

- original_query: The user's input
- corrected_text: The corrected version
- errors: List of error types found
- explanations: Detailed explanations of the corrections
- Technical Details: Each interaction has a unique invocation_id
- Events are timestamped
- The session maintains state and tracks all events


To summarize, this agent:

1. Accepts user input
2. Identifies grammatical and spelling errors
3. Provides corrected versions
4. Explains the corrections
5. Maintains a conversation history
6. The agent is particularly focused on:
7. Spelling corrections (e.g., "ho" → "how", "whut" → "what")
8. Sentence structure (e.g., adding missing "be")
9. Providing educational explanations for each correction

# Hosting Agent on Agent engine

In [49]:
model = "gemini-2.0-flash-001"

from google.genai import types

safety_settings = [
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=types.HarmBlockThreshold.OFF,
    ),
]
generate_content_config = types.GenerateContentConfig(
    safety_settings=safety_settings,
    temperature=0.28,
    max_output_tokens=1000,
    top_p=0.95,
)


In [50]:
from google.adk.agents import Agent

agent = Agent(
    model=model,                                      # Required.
    name='currency_exchange_agent',                   # Required.
    generate_content_config=generate_content_config,  # Optional.
)
app = AdkApp(agent=agent)
for event in app.stream_query(
    user_id="ysian",  # Required
    message="What is the exchange rate from US dollars to Swedish currency?",
):
    print(event)

{'content': {'parts': [{'text': "To give you the most accurate exchange rate, I need to know if you're looking for the rate to Swedish Krona (SEK). Also, please specify if you want the current real-time exchange rate, or if you're interested in a historical rate.\n\nAssuming you want the current USD to SEK exchange rate, I can tell you that 1 US Dollar is currently equivalent to approximately 10.85 Swedish Krona.\n\nPlease note that exchange rates constantly fluctuate, so this is an approximate value. For the most up-to-date and precise rate, I recommend checking a reliable online converter or your bank.\n"}], 'role': 'model'}, 'invocation_id': 'e-debe6ade-664a-48ed-8b08-bd9191e454b4', 'author': 'currency_exchange_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': '7Aw6eJMl', 'timestamp': 1748497148.84866}


## Enable Tracability

In [51]:
app = AdkApp(
    agent=agent,
    enable_tracing=True, # <- new
)

for event in app.stream_query(
    user_id="saurabh",
    message="What is the exchange rate from US dollars to Swedish currency on 2025-04-03?",
):
    print(event)

{'content': {'parts': [{'text': "I can't provide you with a specific exchange rate for USD to SEK on April 3, 2025. Exchange rates fluctuate constantly based on market conditions. You'll need to check a reliable source closer to that date.\n\nHere's where you can find the exchange rate on or closer to April 3, 2025:\n\n*   **Financial Websites:** Reputable financial websites like Google Finance, Yahoo Finance, Bloomberg, or a specific bank's website will provide real-time or near real-time exchange rates.\n*   **Currency Converter Tools:** Many online currency converter tools (like those on the websites mentioned above) will give you the current exchange rate.\n*   **Your Bank or Financial Institution:** Your own bank or financial institution will give you the exchange rate they are using if you plan to exchange currency through them.\n\nRemember to check the exchange rate close to the date you need it for the most accurate information."}], 'role': 'model'}, 'invocation_id': 'e-beda25b

## Agent Engine hosting

In [52]:
remote_app = agent_engines.create(
    app,
    requirements=["google-cloud-aiplatform[agent_engines,adk]>=1.88"],
    # requirements=["google-cloud-aiplatform[agent_engines,adk]],
)

Identified the following requirements: {'cloudpickle': '3.0.0', 'pydantic': '2.9.2', 'google-cloud-aiplatform': '1.94.0'}


INFO:vertexai.agent_engines:Identified the following requirements: {'cloudpickle': '3.0.0', 'pydantic': '2.9.2', 'google-cloud-aiplatform': '1.94.0'}


The following requirements are missing: {'cloudpickle', 'pydantic'}


The following requirements are appended: {'cloudpickle==3.0.0', 'pydantic==2.9.2'}


INFO:vertexai.agent_engines:The following requirements are appended: {'cloudpickle==3.0.0', 'pydantic==2.9.2'}


...

INFO:vertexai.agent_engines:AgentEngine created. Resource name: projects/255766800726/locations/us-central1/reasoningEngines/6651579155114622976


To use this AgentEngine in another session:


INFO:vertexai.agent_engines:To use this AgentEngine in another session:


agent_engine = vertexai.agent_engines.get('projects/255766800726/locations/us-central1/reasoningEngines/6651579155114622976')


INFO:vertexai.agent_engines:agent_engine = vertexai.agent_engines.get('projects/255766800726/locations/us-central1/reasoningEngines/6651579155114622976')


In [53]:
import pprint

pprint.pprint(remote_app.operation_schemas())

[{'api_mode': '',
  'description': 'Get a session for the given user.\n'
                 '\n'
                 '        Args:\n'
                 '            user_id (str):\n'
                 '                Required. The ID of the user.\n'
                 '            session_id (str):\n'
                 '                Required. The ID of the session.\n'
                 '            **kwargs (dict[str, Any]):\n'
                 '                Optional. Additional keyword arguments to '
                 'pass to the\n'
                 '                session service.\n'
                 '\n'
                 '        Returns:\n'
                 '            Session: The session instance (if any). It '
                 'returns None if the\n'
                 '            session is not found.\n'
                 '\n'
                 '        Raises:\n'
                 '            RuntimeError: If the session is not found.\n'
                 '        ',
  'name': 'get

# Agent Features

## Basic Agent

In [54]:
basic_agent = Agent(
    name='basic_agent',
    description='A helpful assistant for user.',
    model='gemini-2.0-flash-001',
    instruction="Answer user's question with your best knowledge.",
)

In [55]:
runner = create_runner(basic_agent)
session = await run_session(content_text('Hello!'), runner=runner)
session = await run_session(content_text('Tell me a joke under 30 words?'), runner=runner, session=session)

User: Hello!
----------------------------------
Author: basic_agent
Content Text: Hello there! How can I help you today?

Event: {'content': {'parts': [{'text': 'Hello there! How can I help you today?\n'}], 'role': 'model'}, 'invocation_id': 'e-99dc1a4b-b80a-4e0a-afc0-2b85bbe534e1', 'author': 'basic_agent', 'id': 'kUzvNzJU', 'timestamp': 1748497311.177459}
----------------------------------
User: Tell me a joke under 30 words?
----------------------------------
Author: basic_agent
Content Text: Why don't scientists trust atoms?

Because they make up everything!

Event: {'content': {'parts': [{'text': "Why don't scientists trust atoms?\n\nBecause they make up everything!\n"}], 'role': 'model'}, 'invocation_id': 'e-920967e1-5ca0-4e4e-a6f5-4a15b5e3c9c1', 'author': 'basic_agent', 'id': 'WF66zJHk', 'timestamp': 1748497312.175247}
----------------------------------


### Agent declaration

In [56]:
sub_agent_1 = Agent(
    name='sub_agent_1',
    description='No.1 sub agent.',
    model='gemini-2.0-flash-001',
    instruction="JUST SAY 1.",
)

sub_agent_2 = Agent(
    name='sub_agent_2',
    description='No.2 sub agent.',
    model='gemini-2.0-flash-001',
    instruction="JUST SAY 2.",
)

## Sequential Agent

In [57]:
sequential_agent = SequentialAgent(
    name='sequential_agent',
    sub_agents=[sub_agent_1, sub_agent_2],
)

In [58]:
runner = create_runner(sequential_agent)
session = await run_session(content_text('Hello!'), runner=runner)

# Expect output:
"""
Author: child_1_agent
Content Text: 1.

...

Author: child_2_agent
Content Text: 2.
"""

User: Hello!
----------------------------------
Author: sub_agent_1
Content Text: 1.

Event: {'content': {'parts': [{'text': '1.\n'}], 'role': 'model'}, 'invocation_id': 'e-4ccd8207-280c-44a0-b514-26dfeb84feff', 'author': 'sub_agent_1', 'id': 'mdN4k513', 'timestamp': 1748497313.108926}
Author: sub_agent_2
Content Text: 2.

Event: {'content': {'parts': [{'text': '2.\n'}], 'role': 'model'}, 'invocation_id': 'e-4ccd8207-280c-44a0-b514-26dfeb84feff', 'author': 'sub_agent_2', 'id': 'EtYbS8En', 'timestamp': 1748497313.991849}
----------------------------------


'\nAuthor: child_1_agent\nContent Text: 1.\n\n...\n\nAuthor: child_2_agent\nContent Text: 2.\n'

## Parallel Agent

In [59]:
from typing import Optional, Any

APP_NAME = 'test_app'
USER_ID = 'test_user'

def create_runner(agent: BaseAgent):
  return InMemoryRunner(agent, app_name=APP_NAME)

def _content_to_text(content: types.Content | None) -> str:
  if not content or not content.parts:
    return ''
  return ''.join([p.text or '' for p in content.parts])

def print_event(event: Event):
  print(f'Author: {event.author}')
  print(f'Content Text: {_content_to_text(event.content)}')
  print(f'Event: {event.model_dump(exclude_none=True, exclude_defaults=True)}')

async def run_session(new_message: types.Content, *, runner: Runner, session: Optional[Session] = None, state: Optional[dict[str, Any]] = None) -> Session:
  if session is None:
    session = runner.session_service.create_session(app_name=APP_NAME, user_id=USER_ID, state=state)

  print(f'User: {_content_to_text(new_message)}')
  print('----------------------------------')
  async for e in runner.run_async(user_id=USER_ID, session_id=session.id, new_message=new_message):
    print_event(e)
  print('----------------------------------')

  session = runner.session_service.get_session(app_name=session.app_name, user_id=session.user_id, session_id=session.id)
  return session

def content_text(msg: str) -> types.Content:
  return types.UserContent(parts=[types.Part(text=msg)])

In [60]:
def roll_die(sides: int) -> int:
  """Roll a die and return the rolled result.

  Args:
    sides: The integer number of sides the die has.

  Returns:
    An integer of the result of rolling the die.
  """
  return random.randint(1, sides)


single_agent_1 = Agent(
    name='async_agent_1',
    model='gemini-2.0-flash-001',
    instruction='Just roll a dice of 8 sides when being asked to roll a die.',
    disallow_transfer_to_parent=False,
    disallow_transfer_to_peers=False,
    tools=[roll_die],
)
single_agent_2 = Agent(
    name='async_agent_2',
    model='gemini-2.0-flash-001',
    instruction='Just roll a dice of 10 sides when being asked to roll a die.',
    disallow_transfer_to_parent=False,
    disallow_transfer_to_peers=False,
    tools=[roll_die],
)
single_agent_3 = Agent(
    name='async_agent_3',
    model='gemini-2.0-flash-001',
    instruction='Just roll a dice of 15 sides when being asked to roll a die.',
    disallow_transfer_to_parent=False,
    disallow_transfer_to_peers=False,
    tools=[roll_die],
)
parallel_agent = ParallelAgent(
    name='parallel_agent',
    sub_agents=[
        single_agent_1,
        single_agent_2,
        single_agent_3,
    ],
)


In [61]:
runner = create_runner(parallel_agent)
session = await run_session(content_text('Hello!'), runner=runner)
session = await run_session(content_text('Roll a dice of 100 sides and tell me what you got.'), runner=runner)

User: Hello!
----------------------------------


ERROR:opentelemetry.context:Failed to detach context
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/opentelemetry/context/__init__.py", line 155, in detach
    _RUNTIME_CONTEXT.detach(token)
  File "/opt/conda/lib/python3.10/site-packages/opentelemetry/context/contextvars_context.py", line 53, in detach
    self._current_context.reset(token)
ValueError: <Token var=<ContextVar name='current_context' default={} at 0x7f8f82389260> at 0x7f8f74653100> was created in a different Context
ERROR:opentelemetry.context:Failed to detach context
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/opentelemetry/context/__init__.py", line 155, in detach
    _RUNTIME_CONTEXT.detach(token)
  File "/opt/conda/lib/python3.10/site-packages/opentelemetry/context/contextvars_context.py", line 53, in detach
    self._current_context.reset(token)
ValueError: <Token var=<ContextVar name='current_context' default={} at 0x7f8f82389260> at 0x7f8f7

Author: async_agent_1
Content Text: Hello! How can I help you today?

Event: {'content': {'parts': [{'text': 'Hello! How can I help you today?\n'}], 'role': 'model'}, 'invocation_id': 'e-10cbcba8-cdfb-4e36-9986-3d8282c4a6cb', 'author': 'async_agent_1', 'branch': 'parallel_agent.async_agent_1', 'id': 'FJqPp1fL', 'timestamp': 1748497314.954083}


ERROR:opentelemetry.context:Failed to detach context
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/opentelemetry/context/__init__.py", line 155, in detach
    _RUNTIME_CONTEXT.detach(token)
  File "/opt/conda/lib/python3.10/site-packages/opentelemetry/context/contextvars_context.py", line 53, in detach
    self._current_context.reset(token)
ValueError: <Token var=<ContextVar name='current_context' default={} at 0x7f8f82389260> at 0x7f8f74696cc0> was created in a different Context
ERROR:opentelemetry.context:Failed to detach context
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/opentelemetry/context/__init__.py", line 155, in detach
    _RUNTIME_CONTEXT.detach(token)
  File "/opt/conda/lib/python3.10/site-packages/opentelemetry/context/contextvars_context.py", line 53, in detach
    self._current_context.reset(token)
ValueError: <Token var=<ContextVar name='current_context' default={} at 0x7f8f82389260> at 0x7f8f7

Author: async_agent_2
Content Text: Hello! How can I help you today?

Event: {'content': {'parts': [{'text': 'Hello! How can I help you today?\n'}], 'role': 'model'}, 'invocation_id': 'e-10cbcba8-cdfb-4e36-9986-3d8282c4a6cb', 'author': 'async_agent_2', 'branch': 'parallel_agent.async_agent_2', 'id': 'jyaAB8gL', 'timestamp': 1748497314.964211}
Author: async_agent_3
Content Text: Hello! How can I assist you today?

Event: {'content': {'parts': [{'text': 'Hello! How can I assist you today?\n'}], 'role': 'model'}, 'invocation_id': 'e-10cbcba8-cdfb-4e36-9986-3d8282c4a6cb', 'author': 'async_agent_3', 'branch': 'parallel_agent.async_agent_3', 'id': 'BLDJgBMY', 'timestamp': 1748497314.977084}
----------------------------------
User: Roll a dice of 100 sides and tell me what you got.
----------------------------------


...

Author: async_agent_2
Content Text: 
Event: {'content': {'parts': [{'function_call': {'id': 'adk-7c19ccd6-40a3-4a08-9853-45ba07836a00', 'args': {'sides': 100}, 'name': 'roll_die'}}], 'role': 'model'}, 'invocation_id': 'e-710b923f-7c89-421d-9bf8-4d662209ef36', 'author': 'async_agent_2', 'long_running_tool_ids': set(), 'branch': 'parallel_agent.async_agent_2', 'id': 'tPU3Bz1T', 'timestamp': 1748497316.27574}
Author: async_agent_2
Content Text: 
Event: {'content': {'parts': [{'function_response': {'id': 'adk-7c19ccd6-40a3-4a08-9853-45ba07836a00', 'name': 'roll_die', 'response': {'result': 8}}}], 'role': 'user'}, 'invocation_id': 'e-710b923f-7c89-421d-9bf8-4d662209ef36', 'author': 'async_agent_2', 'branch': 'parallel_agent.async_agent_2', 'id': 'ofaBJJOE', 'timestamp': 1748497317.560279}
Author: async_agent_3
Content Text: 
Event: {'content': {'parts': [{'function_call': {'id': 'adk-278132bf-1d68-43c6-a494-85279b61e10e', 'args': {'sides': 100}, 'name': 'roll_die'}}], 'role': 'model'}, 'inv

ERROR:opentelemetry.context:Failed to detach context
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/opentelemetry/context/__init__.py", line 155, in detach
    _RUNTIME_CONTEXT.detach(token)
  File "/opt/conda/lib/python3.10/site-packages/opentelemetry/context/contextvars_context.py", line 53, in detach
    self._current_context.reset(token)
ValueError: <Token var=<ContextVar name='current_context' default={} at 0x7f8f82389260> at 0x7f8f74a51a80> was created in a different Context
ERROR:opentelemetry.context:Failed to detach context
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/opentelemetry/context/__init__.py", line 155, in detach
    _RUNTIME_CONTEXT.detach(token)
  File "/opt/conda/lib/python3.10/site-packages/opentelemetry/context/contextvars_context.py", line 53, in detach
    self._current_context.reset(token)
ValueError: <Token var=<ContextVar name='current_context' default={} at 0x7f8f82389260> at 0x7f8f7

Author: async_agent_1
Content Text: I rolled a 100-sided die and got 43.

Event: {'content': {'parts': [{'text': 'I rolled a 100-sided die and got 43.\n'}], 'role': 'model'}, 'invocation_id': 'e-710b923f-7c89-421d-9bf8-4d662209ef36', 'author': 'async_agent_1', 'branch': 'parallel_agent.async_agent_1', 'id': 'BvTUH1Ps', 'timestamp': 1748497317.357248}
Author: async_agent_2
Content Text: I rolled a 100-sided die and got 8.

Event: {'content': {'parts': [{'text': 'I rolled a 100-sided die and got 8.\n'}], 'role': 'model'}, 'invocation_id': 'e-710b923f-7c89-421d-9bf8-4d662209ef36', 'author': 'async_agent_2', 'branch': 'parallel_agent.async_agent_2', 'id': 'Ad2fclNh', 'timestamp': 1748497317.562569}


ERROR:opentelemetry.context:Failed to detach context
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/opentelemetry/context/__init__.py", line 155, in detach
    _RUNTIME_CONTEXT.detach(token)
  File "/opt/conda/lib/python3.10/site-packages/opentelemetry/context/contextvars_context.py", line 53, in detach
    self._current_context.reset(token)
ValueError: <Token var=<ContextVar name='current_context' default={} at 0x7f8f82389260> at 0x7f8f7468f380> was created in a different Context
ERROR:opentelemetry.context:Failed to detach context
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/opentelemetry/context/__init__.py", line 155, in detach
    _RUNTIME_CONTEXT.detach(token)
  File "/opt/conda/lib/python3.10/site-packages/opentelemetry/context/contextvars_context.py", line 53, in detach
    self._current_context.reset(token)
ValueError: <Token var=<ContextVar name='current_context' default={} at 0x7f8f82389260> at 0x7f8f7

Author: async_agent_3
Content Text: I rolled a 100-sided die and got 63.

Event: {'content': {'parts': [{'text': 'I rolled a 100-sided die and got 63.\n'}], 'role': 'model'}, 'invocation_id': 'e-710b923f-7c89-421d-9bf8-4d662209ef36', 'author': 'async_agent_3', 'branch': 'parallel_agent.async_agent_3', 'id': 'UXcCWexM', 'timestamp': 1748497317.599176}
----------------------------------
